In [2]:
from game import State
#from tensorflow.keras.models import load_model
from pathlib import Path
from threading import Thread
import tkinter as tk

In [3]:

from mcts1000 import mcts1000_action
from mcts2000 import mcts2000_action
from mcts100 import mcts100_action
from mcts200 import mcts200_action

from drawmcts import draw_mcts_action
from game import random_action
from newdraw import draw_mcts_newaction

In [4]:
#model = load_model('./model/best.h5')

In [5]:
class GameUI(tk.Frame):
    def __init__(self,master=None,model=None):
        tk.Frame.__init__(self,master)
        self.master.title('おっせろおおお')
        
        #ゲームを始める準備
        self.state = State()
        
        #行動選択を行うモデルをここに置きます
        self.next_action =mcts2000_action
        
        #画面を作るよ
        self.c = tk.Canvas(self,width = 320,height =320 ,highlightthickness = 0)
        self.c.bind('<Button-1>',self.turn_of_human)
        self.c.pack()
        #描画の更新
        self.on_draw()
    
    def turn_of_human(self,event):
        #ゲーム終了
        if self.state.is_done():
            self.state = State()
            self.on_draw()
            
#         #勝敗のやつをやりたい
#         if self.piece_count(self.pieces)+self.piece_count(self.enemy_pieces) == 64 or self.pass_end:
#             print(self.pieces.count(1),end="")
#             print("対",end="")
#             print(self.pieces.count(0))
#             if self.pieces.count(1) > self.pieces.count(0):
#                 print("先攻よお前の勝ちだ")
#             elif self.pieces.count(1) == self.pieces.count(0):
#                 print("引き分けー")
#             else:
#                 print("後攻の勝ち！")
                
            return
        
        if not self.state.is_first_player():
            return
        
        #クリック下位置を行動に変換する
        x = int(event.x/40)
        y = int(event.y/40)
        if x < 0 or 7 < x or y < 0 or 7 <y:
            return
        action = x + y * 8
        
        #合法手ではないときの対応
        legal_actions = self.state.legal_actions()
        if legal_actions == [64]:
            actions = 64 #パスを選択、二回パスになるからそこで試合終了や
        if action != 64 and not (action in legal_actions):
            return
        
        #次の状態の取得
        self.state = self.state.next(action)
        self.on_draw()
        
        #AIのたーーーーん
        self.master.after(1,self.turn_of_ai)
    
    def turn_of_ai(self):
        if self.state.is_done():
            return
        
        #行動を取得
        action = self.next_action(self.state)
        
        #次の行動を取得する
        self.state = self.state.next(action)
        self.on_draw()
    
    #石を描きます
    def draw_piece(self,index,first_player):
        x = (index%8)*40+7
        y = int(index/8)*40+7
        if first_player:
            self.c.create_oval(x,y,x+30,y+30,width = 1.0,outline = '#000000',fill = '#0d0015')
        else:
            self.c.create_oval(x,y,x+30,y+30,width = 1.0,outline = '#000000',fill = '#FFFFFF')
    
    def on_draw(self):
        self.c.delete('all')
        self.c.create_rectangle(0,0,320,320,width = 0.0,fill = '#019a66')
        for i in range(1,8):
            self.c.create_line(0,i*40,320,i*40,width = 0.0,fill = '#000000')
            self.c.create_line(i*40,0,i*40,320,width = 0.0,fill = '#000000')
        for i in range(64):
            if self.state.pieces[i] == 1:
                self.draw_piece(i,self.state.is_first_player())
            if self.state.enemy_pieces[i] == 1:
                self.draw_piece(i,not self.state.is_first_player())

In [6]:
f = GameUI()
f.pack()
f.mainloop() 